In [431]:
import pandas as pd
import numpy as np

## Get the Normalization

In [432]:
from nilearn.image import load_img

In [433]:
comps = [load_img(f"/Users/chris/projects/Nature2023MooneyScripts/results/prediction_error_components/components_{idx}.nii.gz").get_fdata() for idx in range(4)]
comps = [comp[comp != 0] for comp in comps]
comps = [np.linalg.norm(comp) for comp in comps]

## OK

In [434]:
df = pd.read_csv("/Users/chris/projects/Nature2023MooneyScripts/results/prediction_error_components/record.csv")

In [435]:
# Multiply value by the norm to get the actual value
for idx, row in df.iterrows():
    df.loc[idx, "component_value"] = row["component_value"] * comps[int(row["component_idx"])]

## Which image and subject?

In [436]:
# Load one fmri sequence to get the labels
fmri_sequence = pd.read_csv("/Users/chris/projects/Nature2023MooneyScripts/results/fmri_sequences/human_subject4.csv")

# Make a conversion
index_to_image_name = {}
for i, row in fmri_sequence.iterrows():
    index_to_image_name[row["image_index"]] = row["image_path"].split("/")[-1]

In [437]:
pre_threshold = 4
for (subject, image_index), small_df in df.groupby(["subject", "image_index"]):
    # Load the response order
    df_subject = pd.read_excel(f"/Users/chris/projects/Nature2023MooneyScripts/data/fmri_image_order/S{subject}/response_order.xlsx", header=None, names=["image_name", "image_recognized"])

    # Get the image name
    img_name_this = index_to_image_name[image_index]

    # Match with the first row with this image name
    first_row = df_subject[df_subject["image_name"] == img_name_this]

    # Some names are missing a 1
    if len(first_row) == 0:
        img_name_this = img_name_this.replace(".bmp", "1.bmp")
        first_row = df_subject[df_subject["image_name"] == img_name_this]

    # Remove trailing 0s
    recognitions = first_row["image_recognized"].values
    pre_rec = recognitions[:6].mean() >= pre_threshold

    index = small_df.index
    if pre_rec:
        df.loc[index, "reliability"] = -1
    else:
        df.loc[index, "reliability"] = recognitions[6:].mean()

In [438]:
df = df[df["reliability"] != -1]
df = df[df["reliability"] > 1/6]

In [439]:
df.head()

,subject,component_idx,component_value,image_index,reliability
0,4,0,6.293413,0,0.833333
1,4,1,16.764539,0,0.833333
2,4,2,18.239905,0,0.833333
3,4,3,38.393387,0,0.833333
4,4,0,10.538675,1,1.000000


In [440]:
# Turn long into wide: component_value_0, component_value_1, component_value_2, becomes columns
df_rotate = pd.pivot_table(df, index=["subject", "image_index", "reliability"], columns="component_idx", values="component_value").reset_index()
df_rotate

component_idx,subject,image_index,reliability,0,1,2,3
0,4,0,0.833333,6.293413,16.764539,18.239905,38.393387
1,4,1,1.000000,10.538675,3.378043,0.861238,5.977484
2,4,2,1.000000,15.040171,1.035298,7.170261,0.000000
3,4,3,1.000000,14.808265,2.872952,13.374916,26.025493
4,4,4,1.000000,6.984482,2.950828,0.369291,0.393837
...,...,...,...,...,...,...,...
577,25,28,1.000000,12.198598,4.391442,10.863241,15.837474
578,25,29,1.000000,17.703335,5.558810,7.125007,7.555032
579,25,30,1.000000,17.937355,0.000000,0.000000,5.009607
580,25,31,1.000000,6.922676,1.455314,13.234120,11.924708


## Try linear model on correlation?

In [441]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [442]:
df_volume = df.copy()

In [443]:
model = smf.mixedlm(
    "component_value ~ reliability:C(component_idx)",
    data=df_volume,
    groups="subject",
)
m2 = model.fit()
m2.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Mixed Linear Model Regression Results
==========================================================================
Model:                 MixedLM     Dependent Variable:     component_value
No. Observations:      2328        Method:                 REML           
No. Groups:            19          Scale:                  158.1042       
Min. group size:       100         Log-Likelihood:         -9199.1225     
Max. group size:       132         Converged:              Yes            
Mean group size:       122.5                                              
--------------------------------------------------------------------------
                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                       10.363    1.671  6.201 0.000  7.087 13.638
reliability:C(component_idx)[0]  8.250    1.835  4.495 0.000  4.652 11.847
reliability:C(component_idx)[1] -5.187    1.835 -2.826 0.005 -8.784 -1.590
reliability:C(component_idx)[2] -1.432    1.835 -0.780 0.435 -5.029  2.165
reliability:C(component_idx)[3]  3.304    1.835  1.800 0.072 -0.293  6.901
subject Var                      1.674    0.080                           
==========================================================================

"""